In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import prepare_dataframe

from sklearn.preprocessing import MinMaxScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

# North Pacific Ocean Tropical Depressions Novel Forecast Model

## ABSTRACT

Building upon the foundational [analysis of the effects of the El Niño-Southern Oscillation (ENSO) on tropical depressions (TDs) in the North Pacific Ocean](https://github.com/StanDobrev11/enso_effect_on_npacific_tds), this current project takes the research a step further by developing a machine learning (ML) forecast model. The aim is to predict the track, intensity, and progression of tropical depressions based on their initial characteristics and ENSO phases.

In the previous project, correlations between ENSO phases and various tropical depression characteristics, such as frequency, intensity, and storm tracks were successfully identified, especially in the NE Pacific region. It was established that El Niño and La Niña events have discernible impacts on where and how frequently TDs form, as well as their overall intensity across the North Pacific region. This prior work provided valuable insights into how different ENSO conditions influence tropical depressions, laying a strong foundation for more advanced predictive modeling.

## Introduction

### Brief explanation of ENSO and TDs

The **El Niño-Southern Oscillation (ENSO)** is a climate phenomenon characterized by fluctuations in sea surface temperatures and atmospheric conditions in the Pacific Ocean. ENSO consists of three phases: **El Niño**, when ocean waters in the central and eastern Pacific are warmer than average; **La Niña**, when these waters are cooler than normal; and a **Neutral** phase when conditions are closer to the long-term average. ENSO has significant global impacts, affecting weather patterns, rainfall, and storm activity.

A **Tropical Depression (TD)** is a low-pressure weather system that typically develops around 5 degrees north or south of the equator. These systems are the early stage of tropical cyclones and can intensify into stronger storms, such as tropical storms or hurricanes, depending on favorable atmospheric and oceanic conditions. Key factors influencing the development of TDs include sea surface temperature, wind shear, latent heat, moisture, ENSO phase and the Coriolis effect, which helps initiate the system's rotation.

The current project focuses on the development of a machine learning-based forecast model designed to predict the future position, wind speed, and pressure of newly formed tropical depressions at 6-hour intervals. By leveraging historical tropical depression data and known ENSO phases, the model aims to predict the following:

- **TD Track**: The latitudinal and longitudinal coordinates of the storm’s future location.
- **Wind Speed**: Changes in the storm’s wind intensity.
- **Minimum Central Pressure**: Evolution of the storm’s pressure, which is critical for determining its potential strength.
- **Velocity and Direction**: Predict the movement speed and direction of the tropical depression.

Furthermore, as the tropical depression develops, newly acquired data will be incorporated into the model to improve the accuracy of forecasting its further evolution.

### Goals

1. The project aims to develope a series of machine learning models that will predict the future location, wind speed, pressure, and velocity of a tropical depression for the next 6, 12, 18, and 24 hours.
2. Compare the accuracy of different ML models to determine which performs best across varying prediction intervals, with the expectation that the 6-hour prediction will be the most reliable and the 24-hour forecast less so.
3. Evaluate the performance of the models in terms of bias, variance, and overall error using metrics like Mean Squared Error (MSE) and R-squared scores.
4. Optimize the models through grid search and other tuning methods to ensure the best possible performance.

### Steps to be Followed

1. **Data Preparation and Feature Engineering**: The datasets from the previous project will be used. Some of the features will be transformed and scaled, others will be engineered.

2. **Model Selection, Training and Testing**: The machine learning algorithms to be evaluated are Random Forest Regressor and Gradient Boosting Models (XGBoost). The dataset will be split into training and test sets, with cross-validation to ensure model robustness. Separate models will be trained for 6, 12, 18, and 24-hour prediction intervals, with the recursive use of previous predictions for longer intervals.

4. **Model Evaluation**: The performance of the models will be evaluated using metrics such as MSE, R-squared, and bias-variance tradeoffs. The accuracy of the predictions will be analyzed, particularly regarding the 6-hour and 24-hour predictions, with an expectation of decreasing accuracy for longer intervals.

5. **Optimization**: Hyperparameter tuning will be performed using grid search or randomized search to improve the models’ performance, especially in reducing prediction errors.

## Data Preparation and Feature Engineering

### Data Sources

The data used in this project comes from multiple reputable sources, ensuring comprehensive coverage of both tropical depression (TD) activity and environmental factors like sea surface temperature (SST) anomalies:

The SST anomaly data was obtained from the NASA Earth Data AQUA MODIS satellite, providing high-resolution sea surface temperature measurements. These anomalies reflect deviations from normal sea temperatures, which are critical for understanding the effects of El Niño and La Niña events. The data is in .nc format

Data for tropical depressions in the Northwest Pacific was sourced from the Japan Meteorological Agency (JMA). This dataset includes information on each tropical depression's intensity, minimum pressure, wind speeds, latitude, and longitude.

The National Hurricane Center (NHC) provided tropical depression data for the Northeast and Central Pacific, which is similar to the JMA data in terms of recorded metrics like wind speed, pressure, and storm tracks.

The ENSO phases (El Niño, La Niña, and Neutral) were derived from the ONI table provided by the Climate Prediction Center (CPC). The ONI is calculated as the rolling three-month mean SST anomaly in the Niño 3.4 region and is commonly used to identify ENSO phases.

The GSHHS (Global Self-consistent, Hierarchical, High-resolution Shoreline Database) was also utilized in this project to provide accurate geographical boundaries and coastlines for visualizing tropical depression tracks. This dataset offers detailed representations of global shorelines, which are essential for plotting storm trajectories in relation to landmasses and understanding potential landfall locations. GSHHS data ensures that the geospatial analysis of tropical depressions remains accurate and visually consistent when overlaying storm tracks on global maps.

The dataset from the previous project, which includes key information such as TD latitude, longitude, wind speed, minimum central pressure, and ENSO phase, will be further refined. Data cleaning steps included handling missing values, standardizing units, and aligning time formats. The full details of the data cleaning process can be found in the GitHub repository [here](https://github.com/StanDobrev11/enso_effect_on_npacific_tds), where the code for preprocessing is documented and accessible. This ensures the dataset is ready for further analysis and machine learning modeling.

Remaining missing wind data for the JMA dataset will be generated using a Random Forest model, and similarly, missing pressure data for the NHC dataset will be completed using the same method. Additional derived features, such as velocity and direction, will be included, with necessary transformations like trigonometric calculations for geographical coordinates. Features such as latitude, longitude, ENSO phase (one-hot encoded), wind speed, pressure, and the derived velocity and direction will be used to train the models, ensuring a comprehensive representation of tropical depression dynamics.

### Data Preparation

For data preparation, the function **prepare_dataframe()** is imported. Clear explanation of the transformation, performed on the dataframe, are described on the function's docstring. First, we will fill out the missing **wind** values from the JMA dataset.

In [3]:
# read jma data
jma = pd.read_csv('data/csv_ready/jma_td.csv', index_col=0)

# drop first row because first and second are same
jma = jma.drop(jma.index[0])

# add values for the wind, assuming minimum wind of the TD to be 35kn
jma.loc[jma['min_pressure_mBar'] >= 980, 'max_wind_kn'] = 35

In [4]:
jma = prepare_dataframe(jma)

In this project, Cartesian coordinates are used to transform the geographical latitude and longitude into a format that is easier for machine learning models to process. Latitude and longitude represent curved surfaces on Earth, but Cartesian coordinates allow the model to work with a flat, linear representation, making it easier to capture spatial relationships in a way that ML algorithms can interpret effectively. This helps improve the accuracy of the predictions.

Transforming the direction into sine and cosine components is done to avoid discontinuities that arise from using angles directly (since 0° and 360° are mathematically the same direction but numerically far apart). By using sine and cosine, the model can treat direction as a smooth, continuous feature, which better represents the cyclical nature of angles and helps improve learning efficiency.

To obtain velocity, the distance between two consecutive points is calculated using the Mercator projection, specifically following the rhumb line (loxodrome) method, as the distances between observations are relatively short. The rhumb line provides a simpler and more accurate measure for short distances since it maintains a constant bearing between points.

In [5]:
jma.head()

,name,lat,lon,max_wind_kn,min_pressure_mBar,enso,velocity_kn,direction_deg,direction_sin,direction_cos,x,y,z
date,,,,,,,,,,,,,
1951-02-19 12:00:00,UNNAMED,20.0,138.5,35,1010,-1,0.000000,0.000000,0.000000,1.000000,-0.703788,0.622659,0.342020
1951-02-19 18:00:00,UNNAMED,23.0,142.1,35,1000,-1,46.768623,48.146357,0.744852,0.667230,-0.726356,0.565453,0.390731
1951-02-20 00:00:00,UNNAMED,25.0,146.0,35,994,-1,42.077697,60.690506,0.871988,0.489527,-0.751363,0.506801,0.422618
1951-02-20 06:00:00,UNNAMED,27.6,150.6,35,994,-1,50.822108,57.766058,0.845877,0.533377,-0.772073,0.435041,0.463296
1951-02-20 12:00:00,UNNAMED,28.9,153.3,35,994,-1,28.131805,61.338839,0.877471,0.479629,-0.782115,0.393363,0.483282


In [6]:
# the result of this line shows how many observation are missing wind value
jma.max_wind_kn[jma.max_wind_kn == 0].count()

6743

In [7]:
# adding concecutive count of the TDs in order to be able to group and split the datasets
new_td_starts = (jma['velocity_kn'] == 0) & (jma['direction_deg'] == 0)
jma['id'] = new_td_starts.cumsum()

In [8]:
# separate the dataframe
available_data_jma = jma[jma['max_wind_kn'] > 0]
predicted_data_jma = jma[jma['max_wind_kn'] == 0]

X_raw_jma = available_data_jma[['min_pressure_mBar', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]
y_raw_jma = available_data_jma['max_wind_kn']

# splitting the data 80/20 sounds reasonable. There are abt 62000 valid observations
X_train_jma, X_test_jma, y_train_jma, y_test_jma = train_test_split(X_raw_jma, y_raw_jma, test_size=0.2, random_state=42)
X_pred_jma = predicted_data_jma[['min_pressure_mBar', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]

In [9]:
# building the pipeline
preprocessor_jma = ColumnTransformer([
    ('enso', OneHotEncoder(), ['enso']),
    ('scaler', MinMaxScaler(), ['min_pressure_mBar', 'velocity_kn']),
], remainder='passthrough')

pipeline_jma = Pipeline([
    ('preprocess', preprocessor_jma),  # Apply scaling and encoding
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=97))
])

Splitting the data 80/20 means using 80% of the dataset (about 49,600 observations) for training the model and the remaining 20% (about 12,400 observations) for testing. This is a common practice to ensure the model has enough data to learn from while still reserving a significant portion for evaluating its performance on unseen data. The train_test_split function with random_state=42 ensures that the split is reproducible, meaning the same split can be obtained each time the code is run.

This pipeline automates the process of preparing the data and training the model for tropical depression forecasting.

First, the preprocessor step handles the data transformations: **ENSO phases** (El Niño, La Niña, Neutral) are one-hot encoded to convert the categorical data into a format that the model can understand.
**MinPressure** and **Velocity** are scaled using **Min-Max scaling** to normalize the values, making them easier for the model to interpret.
Next, the **RandomForestRegressor** is applied to the preprocessed data. This model uses multiple decision trees to predict the next values for wind speed, pressure, and coordinates based on the historical data and ENSO phase. The pipeline combines all of these steps, making it easier to train and test the model efficiently.

Then we fit the training data to the pipeline and evaluate the model:

In [10]:
pipeline_jma.fit(X_train_jma, y_train_jma)

C:\Users\Master\miniconda3\envs\venv\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enso', OneHotEncoder(),
                                                  ['enso']),
                                                 ('scaler', MinMaxScaler(),
                                                  ['min_pressure_mBar',
                                                   'velocity_kn'])])),
                ('regressor', RandomForestRegressor(random_state=97))])

In [11]:
pipeline_jma.score(X_test_jma, y_test_jma)

0.9902244495798731

In [12]:
y_pred = pipeline_jma.predict(X_test_jma)
print(f'Mean Squared Error: {mean_squared_error(y_test_jma, y_pred)}')

Mean Squared Error: 3.2932354939258315


The result shows that the RandomForestRegressor model is performing very well on the test data, with a high accuracy score of 0.99 (which indicates the model is correctly predicting 99% of the variance in the data). The Mean Squared Error (MSE) of 3.29 also indicates that, on average, the model's predictions deviate by about 3.29 units (kn) from the actual values. This is a low error value, further suggesting that the model is making precise predictions. The value of 3.29 kn deviation in the range of 35 - 140 kn of wind is acceptable.

Further evaluation of the model's reliability and potential overfitting will not be conducted, as the model's output is intended solely for a one-time use to fill missing data in the dataset.

With that in mind, missing wind data will be now applied to the jma dataset and will check if there are any '0' values remaining.

In [13]:
predicted_winds = pipeline_jma.predict(X_pred_jma)
jma.loc[jma['max_wind_kn'] == 0, 'max_wind_kn'] = predicted_winds.astype(int)

In [14]:
jma.max_wind_kn[jma.max_wind_kn == 0].count()

0

In [15]:
importances = pipeline_jma.named_steps['regressor'].feature_importances_
feature_names = pipeline_jma.named_steps['preprocess'].get_feature_names_out()
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))

                     Feature  Importance
3  scaler__min_pressure_mBar    0.986759
9               remainder__z    0.002826
4        scaler__velocity_kn    0.002306
8               remainder__y    0.002231
7               remainder__x    0.002035
5   remainder__direction_sin    0.001531
6   remainder__direction_cos    0.001447
0              enso__enso_-1    0.000353
2               enso__enso_1    0.000281
1               enso__enso_0    0.000230


The feature importance table shows how much each feature contributes to the model's predictive power. In this case, min_pressure_mBar is the most important feature by far, with a feature importance score of 0.986759. This suggests that the model relies heavily on the minimum central pressure to make accurate predictions. The remaining features, including the x, y, z cartesian coordinates, velocity_kn, direction_sin, direction_cos, and the ENSO phase encoded as one-hot variables, have much lower importance, indicating that their contribution to the model's performance is minimal in comparison. This suggests that the pressure plays a critical role in determining the outcome, while the other features provide only marginal improvements.

A similar operation will be performed on the NHC dataset; however, in this case, the missing values are for minimum central pressure. The pressure values that are to be modeled, have a negative value.

In [16]:
nhc = pd.read_csv('data/csv_ready/ne_pacific_td.csv', index_col=0)
# converting the longitude to standart values [-180:180]
nhc['lon'] = 180 - nhc['lon']

In [17]:
nhc = prepare_dataframe(nhc)

C:\Users\Master\PycharmProjects\MachineLearning\Project\utils.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'velocity_kn'] = velocity
C:\Users\Master\PycharmProjects\MachineLearning\Project\utils.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'direction_deg'] = direction


In [18]:
# adding concecutive count of the TDs in order to be able to group and split the datasets
new_td_starts = (nhc['velocity_kn'] == 0) & (nhc['direction_deg'] == 0)
nhc['id'] = new_td_starts.cumsum()

In [19]:
# building the pipeline
preprocessor_nhc = ColumnTransformer([
    ('enso', OneHotEncoder(), ['enso']),
    ('scaler', MinMaxScaler(), ['max_wind_kn', 'velocity_kn']),
], remainder='passthrough')

pipeline_nhc = Pipeline([
    ('preprocess', preprocessor_nhc),  # Apply scaling and encoding
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=97))
])

In [20]:
# separate the dataframe
available_data_nhc = nhc[nhc['min_pressure_mBar'] > 0]
predicted_data_nhc = nhc[nhc['min_pressure_mBar'] < 0]

X_raw_nhc = available_data_nhc[['max_wind_kn', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]
y_raw_nhc = available_data_nhc['min_pressure_mBar']

# splitting the data 80/20 sounds reasonable. There are abt 62000 valid observations
X_train_nhc, X_test_nhc, y_train_nhc, y_test_nhc = train_test_split(X_raw_nhc, y_raw_nhc, test_size=0.2, random_state=97)
X_pred_nhc = predicted_data_nhc[['max_wind_kn', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]

In [21]:
pipeline_nhc.fit(X_train_nhc, y_train_nhc)

C:\Users\Master\miniconda3\envs\venv\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enso', OneHotEncoder(),
                                                  ['enso']),
                                                 ('scaler', MinMaxScaler(),
                                                  ['max_wind_kn',
                                                   'velocity_kn'])])),
                ('regressor', RandomForestRegressor(random_state=97))])

In [22]:
pipeline_nhc.score(X_test_nhc, y_test_nhc)

0.9765985684834164

In [23]:
y_pred = pipeline_nhc.predict(X_test_nhc)
print(f'Mean Squared Error: {mean_squared_error(y_test_nhc, y_pred)}')

Mean Squared Error: 7.408631771969968


In [24]:
predicted_pressure = pipeline_nhc.predict(X_pred_nhc)
nhc.loc[nhc['min_pressure_mBar'] < 0, 'min_pressure_mBar'] = predicted_pressure.astype(int)

In [25]:
nhc.min_pressure_mBar[nhc.min_pressure_mBar < 0].count()

0

In [26]:
importances = pipeline_nhc.named_steps['regressor'].feature_importances_
feature_names = pipeline_nhc.named_steps['preprocess'].get_feature_names_out()
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))

                    Feature  Importance
3       scaler__max_wind_kn    0.965945
8              remainder__y    0.008852
9              remainder__z    0.007665
7              remainder__x    0.005094
4       scaler__velocity_kn    0.004649
5  remainder__direction_sin    0.003096
6  remainder__direction_cos    0.003044
0             enso__enso_-1    0.000618
1              enso__enso_0    0.000610
2              enso__enso_1    0.000427


Similar conclusion can be drawn for this dataset.

## Model Selection, Training and Testing

### Model Selection

In the project,  Random Forest and Gradient Boosting (XGBoost/LightGBM) for time series prediction of tropical depression (TD) tracks and intensity are selected due to their strong performance in handling non-linear relationships and structured time series data. These models are well-suited for working with both categorical and continuous features, such as ENSO phase, latitude, longitude, wind speed, and pressure, which are key components of this study.

**Random Forest for Time Series**:

Random Forest is ideal for this project due to its robustness in capturing non-linear relationships between multiple variables like ENSO phases, wind speed, pressure, and geospatial data (latitude and longitude). By incorporating lagged features (previous time steps of lat/lon, wind speed, pressure), Random Forest can predict future locations and intensities of tropical depressions with high accuracy. Additionally, its ability to provide interpretable results through feature importance will allow us to understand which factors are most critical in influencing TD development and movement. However, Random Forest requires careful feature engineering, such as generating lag features and applying trigonometric transformations to latitude/longitude to maintain spatial consistency, transformation of geographical coordinates (latitude and longitude) and direction into sine and cosine components is necessary to avoid treating these as independent variables.

**Gradient Boosting (XGBoost/LightGBM)**:

Gradient Boosting algorithms, particularly XGBoost and LightGBM, offer higher accuracy and efficiency, especially with larger datasets. Given the structured and complex nature of the time series data in this project, including multiple features like wind, pressure, velocity, and geographic location, Gradient Boosting is a powerful method for handling this complexity. It is particularly useful when combined with careful feature engineering, such as lagged features and trigonometric transformations for geographic data. However, XGBoost and LightGBM require more intensive hyperparameter tuning compared to Random Forest, which can be time-consuming but necessary to achieve optimal results. Similar to Random Forest, these models also require careful engineering of lagged features and transformations for latitude and longitude to capture the spatial relationships of tropical depressions.

In [27]:
jma.to_csv('data/csv_ready/jma_final')
nhc.to_csv('data/csv_ready/nhc_final')